# Commercial Property Transactions Data Cleaning and Transformation

In [ ]:
# Imports
import pandas as pd
import re
from ydata_profiling import ProfileReport

In [ ]:
# Configuration
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 1000)  # Increase display width
pd.set_option(
    "display.float_format",
    lambda x: "{:,.0f}".format(x) if isinstance(x, (int, float)) else str(x),
)

In [ ]:
filepath = "./public/data/CommercialTrans_201910 to 202410.csv"
df = pd.read_csv(filepath)

In [ ]:
profile = ProfileReport(df, title="Profiling Report")
profile.to_file("before.html")

Known issues:
- Dataset has 41 (1.2%) duplicate rows
- Need to convert Transacted Price ($) to int
- Need to convert Unit Price ($) to int
- Remove Area (SQFT) and Unit Price ($ PSF) as we measure in meters
- Need to convert Sale Date to date. current format is (month-year): (oct-21)
  - May include Uppercase characters.
- Area (SQM) to float
- Cast Unit Price ($ PSM) to int
- Tenure needs their info extracted. Example: "99 yrs lease commencing from 1997" and "Freehold"
- Floor level needs their info extracted. Example: "01 to 05"
- Project name includes string "n.a"

Notes:
In Type of Area, Strata refers to Condo, HDB, Shops and other apartment.
In Type of Area, Land refers to landed properties.
Should put a tooltip during visualization about this.
Within visual, convert -5 and -1 in floor levels to B5 and B1

In [1]:
print(f"Before removing duplicates: {len(df)}")
df = df.drop_duplicates()
print(f"After removing duplicates: {len(df)}")

Before removing duplicates: 3556
After removing duplicates: 3509


In [ ]:
# Drop redundant columns
columns_to_drop = ["Area (SQFT)", "Unit Price ($ PSF)"]
df = df.drop(columns=columns_to_drop)

In [ ]:
# Casting:

# convert price columns
df["Transacted Price ($)"] = (
    df["Transacted Price ($)"].str.replace("$", "").str.replace(",", "").astype(int)
)
df["Unit Price ($ PSM)"] = (
    df["Unit Price ($ PSM)"].str.replace("$", "").str.replace(",", "").astype(int)
)
# convert area
df["Area (SQM)"] = df["Area (SQM)"].str.replace(",", "").astype(float)

# convert dates
df["Sale Date"] = pd.to_datetime(df["Sale Date"].str.lower(), format="%b-%y")

In [ ]:
def extract_tenure_info(tenure):
    if pd.isna(tenure):
        return "Unknown", None
    elif "freehold" in tenure.lower():
        return "Freehold", None
    else:
        years = int(tenure.split()[0])
        return "Leasehold", years


df[["Tenure Type", "Lease Years"]] = df["Tenure"].apply(
    lambda x: pd.Series(extract_tenure_info(x))
)

In [ ]:
def extract_floor_levels(floor_str):
    if pd.isna(floor_str) or floor_str == "-":
        return pd.NA, pd.NA
    elif floor_str.startswith("B"):
        nums = re.findall(r"B(\d+)\s+to\s+B(\d+)", floor_str)
        if nums:
            return -int(nums[0][0]), -int(nums[0][1])
        return pd.NA, pd.NA
    else:
        nums = re.findall(r"(\d+)\s+to\s+(\d+)", floor_str)
        if nums:
            return int(nums[0][0]), int(nums[0][1])
        return pd.NA, pd.NA


df[["Floor Min", "Floor Max"]] = df["Floor Level"].apply(
    lambda x: pd.Series(extract_floor_levels(x))
)

In [ ]:
def assign_floor_category(row):
    if pd.isna(row["Floor Min"]):
        return "Unknown"
    elif row["Floor Min"] < 0:
        return "Basement"
    elif row["Floor Min"] <= 5:
        return "01-05"
    elif row["Floor Min"] <= 10:
        return "06-10"
    elif row["Floor Min"] <= 15:
        return "11-15"
    elif row["Floor Min"] <= 20:
        return "16-20"
    else:
        return "21+"


df["Floor Category"] = df.apply(assign_floor_category, axis=1)

In [ ]:
columns_to_drop = ["Floor Level", "Tenure"]
df = df.drop(columns=columns_to_drop)

In [ ]:
# Add transaction year and month columns
df["Transaction Year"] = df["Sale Date"].dt.year
df["Transaction Month"] = df["Sale Date"].dt.month

In [ ]:
# Convert string "n.a" to actual NA.
df.loc[df["Project Name"].str.lower().str.contains("n.a"), "Project Name"] = pd.NA

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Profiling Report")
profile.to_file("after.html")

In [2]:
# Save cleaned dataset
output_filepath = "./public/data/clean_property.csv"
df.to_csv(output_filepath, index=False)
print(f"Cleaned dataset saved to: {output_filepath}")
print(f"Number of rows: {len(df)}")
print(f"Number of columns: {len(df.columns)}")

Cleaned dataset saved to: ./public/data/clean_property.csv
Number of rows: 3509
Number of columns: 17


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3556 entries, 0 to 3555
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Project Name          3556 non-null   object
 1   Street Name           3556 non-null   object
 2   Property Type         3556 non-null   object
 3   Transacted Price ($)  3556 non-null   object
 4   Area (SQFT)           3556 non-null   object
 5   Unit Price ($ PSF)    3556 non-null   object
 6   Sale Date             3556 non-null   object
 7   Type of Area          3556 non-null   object
 8   Area (SQM)            3556 non-null   object
 9   Unit Price ($ PSM)    3556 non-null   object
 10  Tenure                3555 non-null   object
 11  Postal District       3556 non-null   int64 
 12  District Name         3556 non-null   object
 13  Floor Level           3556 non-null   object
dtypes: int64(1), object(13)
memory usage: 389.1+ KB
